In [ ]:
import sys
import os
import datetime

base_folder = r'D:\measuring'

if not base_folder in sys.path:
    sys.path.append(base_folder)
execfile(os.path.join(base_folder,r'analysis\scripts\setup_analysis.py'))
execfile(os.path.join(base_folder,r'analysis\scripts\bell\setup_bell_analysis.py'))

%matplotlib qt

In [ ]:
#bs_folder =  r'D:\measuring\data\2015-04-21-lhfbt4'
#bs_folder  =  r'D:\measuring\data\2015-03-18-lhfbt2'
#bs_folder =  r'D:\measuring\data\2015-04-13-lhfbt3'
#bs_folder =  r'D:\measuring\data\2015-06-10_XX'
#bs_folder =  r'D:\measuring\data\2015-05-15-ZZ'#
#bs_folder = r'D:\measuring\data\2015-06-28-lhfbt5'
bs_folder = r'D:\measuring\data\2015-11-16_XX'
output_folder = bs_folder
analysis_fp = bell_data.get_latest_analysis_fp(output_folder, 'total_events')#bell: 20150803090328'#XXpattern ='20150625125254')#
laser_fp = r'Z:\data\20150310\1540_laser_measurement.dat'

In [ ]:
print analysis_fp

In [ ]:
channel=0

In [ ]:
#load laser pulse shape data
d=np.loadtxt(laser_fp, skiprows=10)
ch0_laser=d[:,0]
ch1_laser=d[:,1]
x_laser=np.arange(5400,5600,0.1280)[:-1]

In [ ]:
#fit laser pulse data
ff = common.fit_gauss
y_laser = ch0_laser if channel == 0 else ch1_laser
args=[0.1,10000,5440.73,2]
fitres = fit.fit1d(x_laser, y_laser, ff, *args, fixed = [0],
                   do_print = True, ret = True)
x0_laser = fitres['params_dict']['x0']
A_laser = fitres['params_dict']['A']

In [ ]:
#get the histogram data
get_total_hist = True
if get_total_hist:
    f = h5py.File(analysis_fp,'r')
    bs_total_hist = f['analysis']['bs_total_hist'].value
    bs_hist_stats = f['analysis']['bs_hist_stats'].value
    f.close()
else:
    #here we get a list of files that actually contributed to the data ie had at least one entanglement event
    #bs_fps, lt3_fps, lt4_fps = bell_data.get_unique_bell_fps_from_analysis_file(analysis_fp)
    #fps=bs_fps[-25:]
    fps=[tb.get_msmt_fp(r'D:\measuring\data\2015-06-28-lhfbt5\BS\20150626\171017_Bell_BS_full_BellTheFinal_day1_run1'),
         tb.get_msmt_fp(r'Z:\data\20150626\162236_Bell_BS_SPCORR_ZPL_SPCORR_lt3'),
         tb.get_msmt_fp(r'Z:\data\20150626\162644_Bell_BS_SPCORR_ZPL_SPCORR_lt3'),
         tb.get_msmt_fp(r'Z:\data\20150626\164507_Bell_BS_SPCORR_ZPL_SPCORR_lt4'),
         tb.get_msmt_fp(r'Z:\data\20150626\165758_Bell_BS_SPCORR_ZPL_SPCORR_lt4')]
    bs_total_hist, bs_hist_stats = bell_data.process_bs_hist_stats(fps, settings.bs_process_params)

In [ ]:
#fit laser pulse part of bell data
do_fit=True
hist_binsize = settings.bs_process_params['hist_binsize_ps']*1e-12
x_bell = np.arange(len(bs_total_hist[:,0]))*hist_binsize *1e9 #ns
hist_bell = bs_total_hist[:,0] if channel==0 else bs_total_hist[:,1]
y_bell =hist_bell.astype(np.float64)/np.sum(bs_hist_stats[:,be._cl_bs_last_sync])*1e4 #prob per [100 ns] bin per excitation pulse *1e4
x_range_fltr = (x_bell>=5420.) & (x_bell<5450.)
xf = x_bell[x_range_fltr]
yf = y_bell[x_range_fltr]
x0_laser_bell = xf[np.argmax(yf)]#fitres['params_dict']['x0']
A_laser_bell = np.max(yf)#fitres['params_dict']['A']
x_bg_fltr = (x_bell>=5550) & (x_bell<5600)
a_bg=np.average(y_bell[x_bg_fltr])
if do_fit:
    x_range_fltr = (x_bell>=x0_laser_bell-1.) & (x_bell<x0_laser_bell+1.)
    xf = x_bell[x_range_fltr]
    yf = y_bell[x_range_fltr]
    ff = common.fit_gauss
    args=[a_bg,A_laser_bell,x0_laser_bell,2]
    fitres = fit.fit1d(xf, yf, ff, *args, fixed = [0],
                   do_print = True, ret = True)
    if fitres['success']:
        x0_laser_bell =fitres['params_dict']['x0']
        A_laser_bell = fitres['params_dict']['A']
        ax=plt.subplot(111)
        ax.plot(xf,yf)
        plot_pts=10000
        x_p=np.linspace(min(xf),max(xf),plot_pts)
        if fitres['success']:
            f_p = fitres['fitfunc'](x_p)
            ax.plot(x_p,f_p)
    else:
        print 'fit failed'

print x0_laser_bell, a_bg, A_laser_bell

In [ ]:
#fit laser pulse part of bell data
do_fit=True
hist_binsize = settings.bs_process_params['hist_binsize_ps']*1e-12
x_bell = np.arange(len(bs_total_hist[:,0]))*hist_binsize *1e9 #ns
hist_bell = bs_total_hist[:,0] if channel==0 else bs_total_hist[:,1]
y_bell =hist_bell.astype(np.float64)/np.sum(bs_hist_stats[:,be._cl_bs_last_sync])*1e4 #prob per [100 ns] bin per excitation pulse *1e4
x_range_fltr = (x_bell>=5444.) & (x_bell<5540.)
xf = x_bell[x_range_fltr]
yf = y_bell[x_range_fltr]
x0_bell = xf[np.argmax(yf)]#fitres['params_dict']['x0']
A_bell = np.max(yf)#fitres['params_dict']['A']
x_bg_fltr = (x_bell>=5550) & (x_bell<5600)
a_bg=np.average(y_bell[x_bg_fltr])
if do_fit:
    x_range_fltr = (x_bell>=x0_laser_bell-1.) & (x_bell<x0_laser_bell+1.)
    xf = x_bell[x_range_fltr]
    yf = y_bell[x_range_fltr]
    ff = common.fit_exp_decay_shifted_with_offset
    #A * exp(-(x-x0)/tau) + a
    #['g_a', 'g_A', 'g_tau', 'g_x0']
    args=[a_bg,A_bell,10,x0_bell]
    fitres = fit.fit1d(xf, yf, ff, *args, fixed = [0],
                   do_print = True, ret = True)
    if fitres['success']:
        print fitres['params_dict']['tau']
        ax=plt.subplot(111)
        ax.plot(xf,yf)
        plot_pts=10000
        x_p=np.linspace(min(xf),max(xf),plot_pts)
        if fitres['success']:
            f_p = fitres['fitfunc'](x_p)
            ax.plot(x_p,f_p)
    else:
        print 'fit failed'

In [ ]:
(80./np.sum(bs_hist_stats[:,be._cl_bs_last_sync]) + 63./12314106271)/2.
#
80./(np.sum(bs_hist_stats[:,be._cl_bs_last_time]*1e-12)/60/60)

In [ ]:
#afterpulsing
# this is the conditional probability per [100 ns] bin *1e4 to obtain an afterpulse, assuming a constant afterpulsing rate, over the integrated window 
f = h5py.File(analysis_fp,'r')
afterpulsing = f['analysis']['afterpulsing'].value
f.close()
after_pulse_st_len = settings.after_pulse_process_params['after_pulse_st_len']
after_pulse_bin_length = (after_pulse_st_len*1e-12)/hist_binsize #500
after_prob_00 = float(np.sum(afterpulsing[:,be._cl_bs_afterpulse_00]))/np.sum(afterpulsing[:,be._cl_bs_firstpulse_0])/after_pulse_bin_length*1e4
after_prob_01 = float(np.sum(afterpulsing[:,be._cl_bs_afterpulse_01]))/np.sum(afterpulsing[:,be._cl_bs_firstpulse_0])/after_pulse_bin_length*1e4
after_prob_10 = float(np.sum(afterpulsing[:,be._cl_bs_afterpulse_10]))/np.sum(afterpulsing[:,be._cl_bs_firstpulse_1])/after_pulse_bin_length*1e4
after_prob_11 = float(np.sum(afterpulsing[:,be._cl_bs_afterpulse_11]))/np.sum(afterpulsing[:,be._cl_bs_firstpulse_1])/after_pulse_bin_length*1e4

after_prob_psi_plus =float(np.sum(afterpulsing[:,be._cl_bs_afterpulse_00])+np.sum(afterpulsing[:,be._cl_bs_afterpulse_11]))/(np.sum(afterpulsing[:,be._cl_bs_firstpulse_0])+np.sum(afterpulsing[:,be._cl_bs_firstpulse_1]))/after_pulse_bin_length*1e4
after_prob_psi_min =float(np.sum(afterpulsing[:,be._cl_bs_afterpulse_01])+np.sum(afterpulsing[:,be._cl_bs_afterpulse_10]))/(np.sum(afterpulsing[:,be._cl_bs_firstpulse_0])+np.sum(afterpulsing[:,be._cl_bs_firstpulse_1]))/after_pulse_bin_length*1e4


In [ ]:
print 'AP 00:',np.sum(afterpulsing[:,be._cl_bs_afterpulse_00])
print 'approx error00:', (np.sqrt(np.sum(afterpulsing[:,be._cl_bs_afterpulse_00])))/np.sum(afterpulsing[:,be._cl_bs_firstpulse_0])/after_pulse_bin_length*1e4
print after_prob_00*after_pulse_bin_length
print after_prob_01*after_pulse_bin_length
print after_prob_10*after_pulse_bin_length
print 'AP 11:',np.sum(afterpulsing[:,be._cl_bs_afterpulse_11])
print after_prob_11*after_pulse_bin_length
print 'approx error11:', (np.sqrt(np.sum(afterpulsing[:,be._cl_bs_afterpulse_11])))/np.sum(afterpulsing[:,be._cl_bs_firstpulse_1])/after_pulse_bin_length*1e4


In [ ]:
plt.close('all')
x0_plot = x0_laser_bell
print x0_laser_bell
plot_after_pulsing=True
x_plot = x_bell - x0_plot
x_laser_adj = x_laser - (x0_laser-x0_laser_bell) - x0_plot
y_laser_adj = y_laser*(A_laser_bell/A_laser)

if plot_after_pulsing:
    y_after_pulse_00 = np.ones(len(x_plot))*after_prob_00
    y_after_pulse_11 = np.ones(len(x_plot))*after_prob_11
y_bg = np.ones(len(x_plot))*a_bg

bad_prob_factor = 0.1 # this number we get from Andreas' simulation, ie when the darcount prob is 0.

figure()
ax=plt.subplot(111)
ax.semilogy(x_plot,y_bell/10.,'r', label='measured hist')
ax.semilogy(x_laser_adj, y_laser_adj/bad_prob_factor, label='laser pulse / {:.2f}'.format(bad_prob_factor))
if plot_after_pulsing:
    if channel == 0:
        ax.semilogy(x_plot,y_after_pulse_00/(bad_prob_factor), label='after pulsing 00 / {:.2f}'.format(bad_prob_factor*2))
    else:
        ax.semilogy(x_plot,y_after_pulse_11/(bad_prob_factor), label='after pulsing 11 / {:.2f}'.format(bad_prob_factor*2))
ax.semilogy(x_plot,y_bg/bad_prob_factor, label='background / {:.2f}'.format(bad_prob_factor))
if x0_plot == 0:
    ax.set_xlim(x0_laser_bell-40, x0_laser_bell+150)
else:
    ax.set_xlim(-40, 150)
ax.legend()
ax.grid(True)
ax.set_xlabel('Sync time [ns] + {:.1f} ns'.format(x0_plot))
ax.set_ylabel('Probability x 1e4 per 100 ps bin per shot')
#plt.savefig(os.path.join(bs_folder, 'hist_comparisson_1.pdf'))

In [ ]:
5476-5426

In [ ]:
execfile(r'K:\ns\qt\Diamond\Eigenpapers\15-Bell\SOM\figures\CHSH_vs_things\plot_ro_length_P.py')

In [ ]:
execfile(r'K:\ns\qt\Diamond\Eigenpapers\15-Bell\SOM\figures\CHSH_vs_things\plot_window_start.py')